In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [88]:
def para_data(soup):
    dictt={}
    try:
        deal=[]
        details=soup.find('div',{"class":"product_main_column"})
        details=details.findAll(['h2','p'])
        details
        for n,det in enumerate(details):
            deal.append(str(det))


        deal=' '.join(deal) 
        deal=deal.split("<h2>")
        for d in deal:
            d=d.split("</h2>")
            if len(d)>1:
                dictt.update({d[0]:d[1]})
    except:
        pass

    return dictt
def introduction(soup):
    intro={}
    try:
        
        b=[]
        background=soup.find('h2',text='Introduction').findNext('p').text
        background_next=str(soup.find('h2',text='Introduction').findNext('p').next_sibling.next_sibling)
        b.append(background)
        if "<p>" in background_next:
            b.append(background_next)
        b=' | '.join(b)    
        
        intro.update({"Introduction":b})
    except:
        pass
    return intro
def publications(soup):
    publicat={}
    try:
        
        pubtext_list=[]
        pubalpha_list=[]
        pub=soup.find('div',{"class":"publications product_publications"})
        pub_text=pub.findAll("div",{"class":"col-md-9"})
        pub_alpha=pub.findAll("div",{"class":"col-md-3"})
        for pt in pub_text:
            pubtext_list.append(pt.text)
        pub_alpha    
        for pa in pub_alpha:
            pubalpha_list.append(pa.a["href"])    
        for i in range(len(pub_alpha)):
            if i==10:
                break
            publicat.update({"Publication_"+str(i+1):pubtext_list[i],"Publication_link_"+str(i+1):pubalpha_list[i]})
    except:
        pass
    return publicat    
   
def target_info(soup):
    try:
        target={}
        ti=soup.find('div',{"id":"target-info"})
        target.update({"Target Info":ti.find('p').text})
    except:
        pass
    return target

def assay_details(soup):
    assay_tab={}
    try:
        assay=soup.find('div',{"id":"assay-details"})
        assay=assay.findAll('p')
        a_table=[]
        for at in assay:
            a_table.append(at.text)
        a_table=' | '.join(a_table)
        assay_tab.update({"Assay Details":a_table})
    except:
        pass
    return assay_tab

def image_details(soup):
    images_det={}
    image_details=soup.findAll('div',{"class":"assay-details p-2"})
    for n,i_d in enumerate(image_details):
        try:
            img=i_d.find('img')
        except:
            pass
        #print(img['src'])
        para_d=[]
        try:
            para=i_d.findAll('p')
            for p in para:
                para_d.append(p.text)
            para_d=' | '.join(para_d)
        except:
            pass
        try:
            images_det.update({"Image_"+str(n+1):img['src']})
        except:
            pass
        try:
            images_det.update({"Image_"+str(n+1)+"_description":para_d})
        except:
            imshow=soup.findAll('div',{"class":"active carousel-item"})
            for im in imshow:
                im=im.findAll('img')
                for b in im:
                    images_det.update({b['src']})
                    
    return images_det    
def pro_title(soup):
    title_d={}
    tit=soup.find('h1',{"class":"page-title"}).text
    title_d.update({"Product Title":tit})
    return title_d
    
def pdf_sheet(soup):
    pdf={}
    datasheets=soup.findAll('div',{"class":"float-left file-icon"})
    try:
        for ds in datasheets:
            alpha=ds.find('a')
            alpha=alpha['href']
            para=ds.find('p').text
            pdf.update({para:alpha})
    except:
        pass
    return pdf

def tables(soup):
    tab_inform={}
    table_info=soup.findAll('table')
    for tab in table_info:

        tab=tab.findAll('tr')
        for table_row in tab:
            try:
                tab_head=table_row.find('th').text
                tab_head=tab_head.lower()
                tab_head=tab_head.split()
                tab_head=' '.join(tab_head)
                if ':' in tab_head:
                    tab_head=tab_head.replace(":","")
                if '\n' in tab_head:
                    tab_head=tab_head.replace("\n","")
                if '\t' in tab_head:
                    tab_head=tab_head.replace("\t","")
                if '\r' in tab_head:
                    tab_head=tab_head.replace("\r","")

                
                tab_data=table_row.find('td').text
                tab_inform.update({tab_head:tab_data})
            except:
                pass
    return tab_inform        

def sizes_prices(soup):
    ret_price_list=[]
    try:
        options=[]
        price_data=[]
        o_title=soup.findAll('span',{"class":"option-title"})
        price=soup.findAll('span',{"class":"option-price"})
        for i in o_title:
            i=i.text.replace('\n',"")
            i=i.split()
            i=' '.join(i)
            options.append(i)
        for p in price:
            p=p.text.replace('\t',"")
            p=p.replace('\n',"")
            p=p.split()
            p=' '.join(p)
            price_data.append(p)
        for d in range(len(price_data)):
            price_size={}
            price_size.update({"Size":options[d],"Price":price_data[d]})
            ret_price_list.append(price_size)
    except:
        price=soup.find('span',{"class":"price"}).text
        size={"Price":price}
        ret_price_list.append(size)

    return ret_price_list

def background_description(soup):
    back_knowledge={}
    back=[]
    try:

        background=soup.find('h2',text='Background').findNext('p').text
        background_next=str(soup.find('h2',text='Background').findNext('p').next_sibling.next_sibling)
        if "<p>" in background_next:
            back.append(background)
            back.append(background_next)
        else:
            back.append(background)
        back=' | '.join(back)
        back_knowledge.update({"background":back})
    except:
        pass
    return back_knowledge
    



In [68]:
#https://www.bosterbio.com/catalogsearch/result/index/?p=2&q=HRP+secondary
all_urls_primary_antibody=[]
for i in range(2):
    #="https://www.bosterbio.com/products/primary-antibodies.html?p="+str(i+1)
    s="https://www.bosterbio.com/catalogsearch/result/index/?p="+str(i+1)+"&q=ECL+Enhanced+Chemiluminescent"
    source=requests.get(s).text
    soup=BeautifulSoup(source,'lxml')
    links_check=soup.findAll('a',{"class":"product-item-link"})
    for lnk in links_check:
        all_urls_primary_antibody.append(lnk['href'])
all_urls_primary_antibody    

['https://www.bosterbio.com/enhanced-chemiluminescent-reagent-kit-mouse-igg-for-western-blotting-ek1001-boster.html',
 'https://www.bosterbio.com/enhanced-chemiluminescent-reagent-kit-mouse-igm-for-western-blotting-ek1005-boster.html',
 'https://www.bosterbio.com/enhanced-chemiluminescent-reagent-kit-rat-igg-for-western-blotting-ek1004-boster.html',
 'https://www.bosterbio.com/enhanced-chemiluminescent-reagent-kit-rabbit-igg-for-western-blotting-ek1002-boster.html',
 'https://www.bosterbio.com/enhanced-chemiluminescent-reagent-kit-goat-igg-for-western-blotting-ek1003-boster.html',
 'https://www.bosterbio.com/ecl-plus-western-blotting-substrate-ar1196-boster.html',
 'https://www.bosterbio.com/hypersensitive-wb-chemiluminescent-substrate-cr0001-5-boster.html',
 'https://www.bosterbio.com/ecl-western-blotting-substrate-ar1170-boster.html',
 'https://www.bosterbio.com/hypersensitive-wb-chemiluminescent-substrate-cr0001-10-boster.html',
 'https://www.bosterbio.com/hypersensitive-wb-chemilum

In [69]:
empty=[]
for num,q in enumerate(all_urls_primary_antibody):
    try:
        #url1="https://www.bosterbio.com/catalogsearch/result/?q="+q
        source=requests.get(q).text
        soup=BeautifulSoup(source,'lxml')
        #url2=soup.find('a',{"class":"product-item-link"})['href']
        #source=requests.get(url2).text
        #soup=BeautifulSoup(source,'lxml')
        sizes=sizes_prices(soup)
        tabular_data=tables(soup)
        data_pdf=pdf_sheet(soup)
        tit=pro_title(soup)
        t_i=target_info(soup)
        assay_d=assay_details(soup)
        bc=background_description(soup)
        imgg=image_details(soup)
        pubg=publications(soup)
        intr=introduction(soup)
        for size in sizes:

            if len(tit)>0:
                size.update(tit)
            size.update({"Product URL":q})
            if(len(tabular_data)>0):
                size.update(tabular_data)
            if len(data_pdf)>0:    
                size.update(data_pdf)
            if len(assay_d)>0:    
                size.update(assay_d)
            if len(bc)>0:    
                size.update(bc)
            if len(imgg)>0:    
                size.update(imgg)
            if len(t_i)>0:
                size.update(t_i)
            if len(pubg)>0:
                size.update(pubg)
            if len(intr)>0:
                size.update(intr)    
            empty.append(size)    

        print("process number "+str(num)+" completed")

    except:
        continue


process number 0 completed
process number 1 completed
process number 2 completed
process number 3 completed
process number 4 completed
process number 5 completed
process number 6 completed
process number 7 completed
process number 8 completed
process number 9 completed
process number 10 completed
process number 11 completed
process number 12 completed
process number 13 completed
process number 14 completed
process number 15 completed
process number 16 completed


In [70]:
len(empty)

20

In [71]:
df=pd.DataFrame(empty)
linkk=[]
for i in df[" Datasheet "]:
    if "https:/" in i:
        linkk.append(i)
    else:
        i="https://www.bosterbio.com"+i
        linkk.append(i)
df[" Datasheet "]=linkk
df.columns

Index(['Size', 'Price', 'Product Title', 'Product URL', 'sku', 'size',
       'reactivity', 'clonality', 'clone number', 'host', 'ig isotype',
       'applications', 'product name', 'sku/catalog number',
       'storage & handling', 'description', 'cite this product', 'specificity',
       'contents/buffer', 'form', 'reconstitution', 'immunogen', 'uniprot id',
       'gene name', 'protein name', 'tissue specificity', 'alternative names',
       'subcellular localization', 'molecular weight', 'protein function',
       'research areas', 'background', ' Datasheet ', ' Protocols ', ' COA ',
       ' MSDS ', 'Assay Details', 'Image_1', 'Image_1_description',
       'Target Info', 'Publication_1', 'Publication_link_1', 'Image_2',
       'Image_2_description', 'Image_3', 'Image_3_description', 'Image_4',
       'Image_4_description', 'Image_5', 'Image_5_description'],
      dtype='object')

In [75]:
#df["pack size"].isna().sum()
df.drop(["product name","size","sku/catalog number"],axis=1,inplace=True)
df["sku"].isna().sum()

0

In [76]:
df["Introduction"].isna().sum()
#df["df.drop(["pack size"] ,axis=1,inplace=True)"]
#df[df["background"].isna()]["background"]

KeyError: 'Introduction'

In [77]:
df.to_excel("ECL+Enhanced+Chemiluminescent.xlsx",header=True,index=False)

In [15]:
data=pd.read_excel("C:/Users/omama/Downloads/Data files/boster_bio_primary_antobodies_clean_data.xlsx")
p_code=list(data["Column145"][1:])
p_code

['MA1001;',
 'MA1002;',
 'MA1004;',
 'MA1005;',
 'MA1007;',
 'MA1008;',
 'MA1009;',
 'MA1010;',
 'MA1011;',
 'MA1000;',
 'MA1012;',
 'MA1013;',
 'MA1014;',
 'MA1015;',
 'MA1016;',
 'MA1017;',
 'MA1018;',
 'MA1019;',
 'MA1020;',
 'MA1021;',
 'MA1022;',
 'MA1023;',
 'MA1024;',
 'MA1026;',
 'MA1028;',
 'MA1029;',
 'MA1030;',
 'MA1032;',
 'MA1033;',
 'MA1034;',
 'MA1036;',
 'MA1037;',
 'MA1038;',
 'MA1039;',
 'MA1040;',
 'MA1041;',
 'MA1042;',
 'MA1044;',
 'MA1045;',
 'MA1047;',
 'MA1048;',
 'MA1049;',
 'MA1050;',
 'MA1051;',
 'MA1052;',
 'MA1053;',
 'MA1054;',
 'MA1055;',
 'MA1056;',
 'MA1057;',
 'MA1058;',
 'MA1060;',
 'MA1061;',
 'MA1062;',
 'MA1063;',
 'MA1064;',
 'MA1067;',
 'MA1068;',
 'MA1069;',
 'MA1070;',
 'MA1071;',
 'MA1072;',
 'MA1074;',
 'MA1075;',
 'MA1076;',
 'MA1077;',
 'MA1078;',
 'MA1079;',
 'MA1080;',
 'MA1081;',
 'MA1082;',
 'MA1083;',
 'MA1085;',
 'MA1087;',
 'MA1088;',
 'MA1089;',
 'MA1090;',
 'MA1091;',
 'MA1092;',
 'MA1093;',
 'MA1094;',
 'MA1095;',
 'MA1096;',
 'MA

0


In [5]:
empty=[]
for num,ss in enumerate(lis[10:30]):
    

    s="https://www.bosterbio.com/catalogsearch/result/?q="+str(ss)

    source=requests.get(s).text
    soup=BeautifulSoup(source,'lxml')
    sizes=sizes_prices(soup)
    tabular_data=tables(soup)
    data_pdf=pdf_sheet(soup)
    tit=pro_title(soup)
    t_i=target_info(soup)
    assay_d=assay_details(soup)
    bc=background_description(soup)
    imgg=image_details(soup)
    for size in sizes:

        if len(tit)>0:
            size.update(tit)
        size.update({"Product URL":s})
        if(len(tabular_data)>0):
            size.update(tabular_data)
        if len(data_pdf)>0:    
            size.update(data_pdf)
        if len(assay_d)>0:    
            size.update(assay_d)
        if len(bc)>0:    
            size.update(bc)
        if len(imgg)>0:    
            size.update(imgg)
        if len(t_i)>0:
            size.update(t_i)
        empty.append(size)    

    print("process number "+str(num)+" completed")
    empty    

process number 0 completed
process number 1 completed
process number 2 completed
process number 3 completed
process number 4 completed
process number 5 completed
process number 6 completed
process number 7 completed
process number 8 completed
process number 9 completed
process number 10 completed
process number 11 completed
process number 12 completed
process number 13 completed
process number 14 completed
process number 15 completed
process number 16 completed
process number 17 completed
process number 18 completed
process number 19 completed


[]

In [7]:
source=requests.get("https://www.bosterbio.com/unconjugated-rabbit-anti-rat-igg-secondary-antibody-ba1042-boster.html").text
soup=BeautifulSoup(source,'lxml')


In [10]:
div=soup.findAll("div")
for d in div:
    dd=d.findAll('p')
    dx=d.findAll('h2')
    
    print(dx)

[]
[]
[]
[]
[]
[<h2>Featured Service:</h2>, <h2>Featured Service:</h2>, <h2>Introduction</h2>, <h2>Publications</h2>]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[<h2>Featured Service:</h2>, <h2>Featured Service:</h2>]
[<h2>Featured Service:</h2>, <h2>Featured Service:</h2>]
[]
[<h2>Featured Service:</h2>, <h2>Featured Service:</h2>]
[<h2>Featured Service:</h2>, <h2>Featured Service:</h2>]
[<h2>Featured Service:</h2>, <h2>Featured Service:</h2>]
[<h2>Featured Service:</h2>, <h2>Featured Service:</h2>]
[<h2>Featured Service:</h2>]
[]
[]
[]
[<h2>Featured Service:</h2>]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[<h2>Featured Service:</h2>]
[<h2>Featured Service:</h2>]
[<h2>Featured Service:</h2>]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[<h2>Introduction</h2>, <h2>Publications</h2>]
[<h2>Introduction</h2>, <h2>Publications</h2>]
[<h2>Introduction</h2>]
[]
[]
[]
[]
[]
[]
[<h2>Introductio

In [80]:
#https://www.bosterbio.com/products/western-blot-reagents.html?p=2
all_urls_primary_antibody=[]
for i in range(8):
    #="https://www.bosterbio.com/products/primary-antibodies.html?p="+str(i+1)
    s="https://www.bosterbio.com/products/ihc-reagents.html?p="+str(i+1)
    source=requests.get(s).text
    soup=BeautifulSoup(source,'lxml')
    links_check=soup.findAll('a',{"class":"product-item-link"})
    for lnk in links_check:
        all_urls_primary_antibody.append(lnk['href'])
all_urls_primary_antibody    

['https://www.bosterbio.com/nf-kb-luciferase-reporter-raw264-7-cell-line-rc1993-boster.html',
 'https://www.bosterbio.com/hrp-conjugated-donkey-anti-goat-igg-sabc-kit-sa1048-boster.html',
 'https://www.bosterbio.com/hrp-conjugated-donkey-anti-rabbit-igg-sabc-kit-sa1047-boster.html',
 'https://www.bosterbio.com/hrp-conjugated-donkey-anti-mouse-igg-sabc-kit-sa1040-boster.html',
 'https://www.bosterbio.com/hrp-conjugated-donkey-anti-mouse-igm-sabc-kit-sa1039-boster.html',
 'https://www.bosterbio.com/hrp-conjugated-donkey-anti-rat-igg-sabc-kit-sa1038-boster.html',
 'https://www.bosterbio.com/hrp-conjugated-donkey-anti-mouse-igg-sabc-kit-sa1037-boster.html',
 'https://www.bosterbio.com/casein-based-tbs-blocking-buffer-pack-ar0183-boster.html',
 'https://www.bosterbio.com/casein-based-pbs-blocking-buffer-pack-ar0182-boster.html',
 'https://www.bosterbio.com/ip-lysis-buffer-ar0107-boster.html',
 'https://www.bosterbio.com/nuclear-fast-red-ar0008-boster.html',
 'https://www.bosterbio.com/haema

In [89]:
empty=[]
for num,q in enumerate(all_urls_primary_antibody):
    try:
        #url1="https://www.bosterbio.com/hrp-conjugated-anti-human-igg-sabc-kit-sa2004-boster.html"
        source=requests.get(q).text
        soup=BeautifulSoup(source,'lxml')
        #url2=soup.find('a',{"class":"product-item-link"})['href']
        #source=requests.get(url2).text
        #soup=BeautifulSoup(source,'lxml')
        sizes=sizes_prices(soup)
        if len(sizes)==0:
            sizes=[]
            alter={}
            alternate_price=soup.find("span",{"class":"price"}).text
            alter.update({"Price":alternate_price})
            sizes.append(alter)
            
        tabular_data=tables(soup)
        data_pdf=pdf_sheet(soup)
        tit=pro_title(soup)
        t_i=target_info(soup)
        assay_d=assay_details(soup)
        bc=background_description(soup)
        imgg=image_details(soup)
        if len(imgg)<1:
            imgg={}
            imshow=soup.findAll('div',{"class":"carousel-item"})
            for gg,im in enumerate(imshow):
                im=im.findAll('img')
                for b in im:
                    imgg.update({"Image_"+str(gg+1):b['src']})
        pubg=publications(soup)
        intr=introduction(soup)
        para=para_data(soup)
        for size in sizes:

            if len(tit)>0:
                size.update(tit)
            size.update({"Product URL":q})
            if(len(tabular_data)>0):
                size.update(tabular_data)
            if len(data_pdf)>0:    
                size.update(data_pdf)
            if len(assay_d)>0:    
                size.update(assay_d)
            if len(bc)>0:    
                size.update(bc)
            if len(imgg)>0:    
                size.update(imgg)
            if len(t_i)>0:
                size.update(t_i)
            if len(pubg)>0:
                size.update(pubg)
            if len(intr)>0:
                size.update(intr) 
            if len(para)>0:
                size.update(para)     
            empty.append(size)    

        print("process number "+str(num)+" completed")

    except:
        continue


process number 0 completed
process number 1 completed
process number 2 completed
process number 3 completed
process number 4 completed
process number 5 completed
process number 6 completed
process number 7 completed
process number 8 completed
process number 9 completed
process number 10 completed
process number 11 completed
process number 12 completed
process number 13 completed
process number 14 completed
process number 15 completed
process number 16 completed
process number 17 completed
process number 18 completed
process number 19 completed
process number 20 completed
process number 21 completed
process number 22 completed
process number 23 completed
process number 24 completed
process number 25 completed
process number 26 completed
process number 27 completed
process number 28 completed
process number 29 completed
process number 30 completed
process number 31 completed
process number 32 completed
process number 33 completed
process number 34 completed
process number 35 completed
pr

In [92]:
df=pd.DataFrame(empty)
for i in df.columns:
    #print(i)
    summ=df[i].isna().sum()
    #print(summ)
    print("**********")
    if summ>=80:
        df.drop([i],inplace=True,axis=1)
        
df#.columns    


**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********

,Size,Price,Product Title,Product URL,sku,size,application,physical state,product name,sku/catalog number,...,Notes,Application,Usage and Handling,Chemical Information,List of Components,product type,Image_1_description,Image_2_description,Image_3,Image_3_description
0,For profit,"$1,850.00",\nNF-kB Luciferase Reporter-RAW264.7 Cell Line,https://www.bosterbio.com/nf-kb-luciferase-rep...,AR1068,500 mL,"Preserving Samples, Fixing Cells For Histology...","The solution should be clear, colorless, with...",4% Paraformaldehyde (PFA) Solution in PBS,AR1068,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Non-profit,"$1,375.00",\nNF-kB Luciferase Reporter-RAW264.7 Cell Line,https://www.bosterbio.com/nf-kb-luciferase-rep...,AR1068,500 mL,"Preserving Samples, Fixing Cells For Histology...","The solution should be clear, colorless, with...",4% Paraformaldehyde (PFA) Solution in PBS,AR1068,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,$60.00,\nHRP Conjugated Donkey Anti-Goat IgG SABC Kit,https://www.bosterbio.com/hrp-conjugated-donke...,SA1048,1 kit,NaN,NaN,HRP Conjugated Donkey Anti-Goat IgG SABC Kit ...,SA1048,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,$60.00,\nHRP Conjugated Donkey Anti-Rabbit IgG SABC Kit,https://www.bosterbio.com/hrp-conjugated-donke...,SA1047,1 kit,NaN,NaN,HRP Conjugated Donkey Anti-Rabbit IgG SABC Kit...,SA1047,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,$60.00,\nHRP Conjugated Donkey Anti-Mouse IgG SABC Kit,https://www.bosterbio.com/hrp-conjugated-donke...,SA1040,1 kit,NaN,NaN,HRP Conjugated Donkey Anti-Mouse IgG SABC Kit ...,SA1040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,NaN,$280.00,\nHRP Conjugated anti-Human IgG SABC Kit,https://www.bosterbio.com/hrp-conjugated-anti-...,SA2004,1 kit,NaN,NaN,HRP Conjugated anti-Human IgG SABC Kit ...,SA2004,...,NaN,NaN,NaN,NaN,NaN,NaN,Boster Kit Box,NaN,NaN,NaN
78,NaN,$280.00,\nHRP Conjugated anti-Goat IgG SABC Kit,https://www.bosterbio.com/hrp-conjugated-anti-...,SA2003,1 kit,NaN,NaN,HRP Conjugated anti-Goat IgG SABC Kit ...,SA2003,...,NaN,NaN,NaN,NaN,NaN,NaN,Boster Kit Box,NaN,NaN,NaN
79,NaN,$280.00,\nHRP Conjugated anti-Rabbit IgG SABC Kit,https://www.bosterbio.com/hrp-conjugated-anti-...,SA2002,1 kit,NaN,NaN,HRP Conjugated anti-Rabbit IgG SABC Kit ...,SA2002,...,NaN,NaN,NaN,NaN,NaN,NaN,ZAP70 was detected in paraffin-embedded sectio...,CRM1 was detected in paraffin-embedded section...,NaN,NaN
80,NaN,$280.00,\nHRP Conjugated anti-Mouse IgG SABC Kit,https://www.bosterbio.com/hrp-conjugated-anti-...,SA2001,1 kit,NaN,NaN,HRP Conjugated anti-Mouse IgG SABC Kit ...,SA2001,...,NaN,NaN,NaN,NaN,NaN,NaN,"Episialin, EMA was detected in paraffin-embedd...",NaN,NaN,NaN


In [93]:
linkk=[]
for i in df[" Datasheet "]:
    if "https:/" in i:
        linkk.append(i)
    else:
        i="https://www.bosterbio.com"+i
        linkk.append(i)
df[" Datasheet "]=linkk


In [99]:
df["ku"].isna().sum()

59

In [101]:
df.drop(["product name","Size","pack size","sku/catalog number"],axis=1,inplace=True)


In [102]:
for i in df.columns:
    df[i]=df[i].str.replace("<p>","")
    df[i]=df[i].str.replace("</p>","|")
    df[i]=df[i].str.replace("\n","|")
df    

,Price,Product Title,Product URL,sku,size,application,physical state,form,storage,description,...,Notes,Application,Usage and Handling,Chemical Information,List of Components,product type,Image_1_description,Image_2_description,Image_3,Image_3_description
0,"$1,850.00",|NF-kB Luciferase Reporter-RAW264.7 Cell Line,https://www.bosterbio.com/nf-kb-luciferase-rep...,AR1068,500 mL,"Preserving Samples, Fixing Cells For Histology...","The solution should be clear, colorless, with...",Liquid,Store at -20˚C for one year.,4% Paraformaldehyde solution in PBS is a ready...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"$1,375.00",|NF-kB Luciferase Reporter-RAW264.7 Cell Line,https://www.bosterbio.com/nf-kb-luciferase-rep...,AR1068,500 mL,"Preserving Samples, Fixing Cells For Histology...","The solution should be clear, colorless, with...",Liquid,Store at -20˚C for one year.,4% Paraformaldehyde solution in PBS is a ready...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,$60.00,|HRP Conjugated Donkey Anti-Goat IgG SABC Kit,https://www.bosterbio.com/hrp-conjugated-donke...,SA1048,1 kit,NaN,NaN,"Concentrated, Liquid",NaN,"HRP Conjugated Donkey Anti-Goat IgG SABC Kit, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,$60.00,|HRP Conjugated Donkey Anti-Rabbit IgG SABC Kit,https://www.bosterbio.com/hrp-conjugated-donke...,SA1047,1 kit,NaN,NaN,"Concentrated, Liquid",NaN,HRP Conjugated Donkey Anti-Rabbit IgG SABC Kit...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,$60.00,|HRP Conjugated Donkey Anti-Mouse IgG SABC Kit,https://www.bosterbio.com/hrp-conjugated-donke...,SA1040,1 kit,NaN,NaN,"Concentrated, Liquid",NaN,"HRP Conjugated Donkey Anti-Mouse IgG SABC Kit,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,$280.00,|HRP Conjugated anti-Human IgG SABC Kit,https://www.bosterbio.com/hrp-conjugated-anti-...,SA2004,1 kit,NaN,NaN,NaN,NaN,"HRP Conjugated anti-Human IgG SABC Kit, with t...",...,NaN,NaN,NaN,NaN,NaN,NaN,Boster Kit Box,NaN,NaN,NaN
78,$280.00,|HRP Conjugated anti-Goat IgG SABC Kit,https://www.bosterbio.com/hrp-conjugated-anti-...,SA2003,1 kit,NaN,NaN,NaN,NaN,"HRP Conjugated anti-Goat IgG SABC Kit, with th...",...,NaN,NaN,NaN,NaN,NaN,NaN,Boster Kit Box,NaN,NaN,NaN
79,$280.00,|HRP Conjugated anti-Rabbit IgG SABC Kit,https://www.bosterbio.com/hrp-conjugated-anti-...,SA2002,1 kit,NaN,NaN,NaN,NaN,"HRP Conjugated anti-Rabbit IgG SABC Kit, with ...",...,NaN,NaN,NaN,NaN,NaN,NaN,ZAP70 was detected in paraffin-embedded sectio...,CRM1 was detected in paraffin-embedded section...,NaN,NaN
80,$280.00,|HRP Conjugated anti-Mouse IgG SABC Kit,https://www.bosterbio.com/hrp-conjugated-anti-...,SA2001,1 kit,NaN,NaN,NaN,NaN,"HRP Conjugated anti-Mouse IgG SABC Kit, with t...",...,NaN,NaN,NaN,NaN,NaN,NaN,"Episialin, EMA was detected in paraffin-embedd...",NaN,NaN,NaN


In [103]:
df.to_excel("IHCreagent.xlsx",header=True,index=False)